# Notebook to read PEATMAP data and rasterize it 


Table of content of the notebook
* [Classical import](#import)
* [Path to the different folders and files used across the notebook](#allpaths)
* [Unzip PEATMAP files](#filemanip)
* [Shape file to dataframe](#shp2df) 
* [Authentification GEE](#gee_auth)
    * [Select the shapefile from an given AOI](#select_shp)
    * [Function to plot all the shapefile on a google map](#plotallshp")
    * [Function to check the coverage of the shape files](#chkcoverage)
    * [Function to crop the shape file to the AOI](#cropshp)
* [Rasterisation of the shapefiles](#rastershp)

<a class="anchor" id="select_shp"></a>


**DATA**

Source of the peatmap data : https://archive.researchdata.leeds.ac.uk/251/

Related to the article: 

Xu, J., Morris, P. J., Liu, J., & Holden, J. (2018). PEATMAP: Refining estimates of global peatland distribution based on a meta-analysis. Catena, 160, 134-140. https://doi.org/10.1016/j.catena.2017.09.010




## Classical imports <a class="anchor" id="import"></a>

In [ ]:
import os

from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import geopandas as gpd

# allow images to display in the notebook
from IPython.display import Image

# remove warnings
import warnings
warnings.filterwarnings('ignore')

# import libraries
import json


from libLandsat8 import *
from libPeatland import *

## User parameters

In [ ]:
# Region of interest
AOIn45w80 = {"xmin": -80, "ymin": 45, "xmax": -85, "ymax":  50}
AOIn45w105 = {"xmin": -105.0, "ymin": 45, "xmax": -110.0, "ymax":  50} # Hudson Bay 2
AOIn45w100 = {"xmin": -105.0, "ymin": 45, "xmax": -100.0, "ymax":  50} # Hudson Bay 1


AOIn50w80 = {"xmin": -80, "ymin": 50, "xmax": -85, "ymax":  55}
AOIn50w85 = {"xmin": -85, "ymin": 50, "xmax": -90, "ymax":  55}
AOIn50w95 = {"xmin": -95.0, "ymin": 50, "xmax": -100, "ymax":  55}
AOI3  = {"xmin": -100.0, "ymin": 50, "xmax": -105.0, "ymax":  55} # Hudson Bay
AOI4 = {"xmin": -105.0, "ymin": 50, "xmax": -110.0, "ymax":  55} # Hudson Bay 4

AOIn55w95 = {"xmin": -100, "ymin": 55, "xmax": -95.0, "ymax":  60} # LS8_n55w95
AOI6 = {"xmin": -110, "ymin": 55, "xmax": -105.0, "ymax":  60} # LS8_n55w105
AOIn55w100 = {"xmin": -100, "ymin": 55, "xmax": -105.0, "ymax":  60} # LS8_n55w100
AOIn55w110 = {"xmin": -115, "ymin": 55, "xmax": -110, "ymax":  60} # LS8_n55w110
AOI12 = {"xmin": -95, "ymin": 55, "xmax": -90, "ymax":  60} # LS8_n55w90

AOI8 = {"xmin": -105, "ymin": 60, "xmax": -100.0, "ymax":  65} 
AOI9 = {"xmin": -100, "ymin": 60, "xmax": -95.0, "ymax":  65} 
AOI10 = {"xmin": -95, "ymin": 60, "xmax": -90.0, "ymax":  65} 
AOI11 = {"xmin": -110, "ymin": 60, "xmax": -105.0, "ymax":  65} 
AOI14 = {"xmin": -115, "ymin": 60, "xmax": -110.0, "ymax":  65}
AOIn60w115 = {"xmin": -115, "ymin": 60, "xmax": -120.0, "ymax":  65}

AOIn65w90 = {"xmin": -90, "ymin": 65, "xmax": -95, "ymax":  70}
AOIn65w95 = {"xmin": -95, "ymin": 65, "xmax": -100, "ymax":  70}

#AOI ={"xmin": 58.5, "ymin": 54.8, "xmax": 90.1, "ymax": 70.7} # Russia
#AOI = {"xmin": 24, "ymin": 62.9, "xmax": 29.06, "ymax": 68.1} # Finland


list_AOI = [AOIn45w105, AOIn45w100, AOI3, AOI4, AOIn55w95, AOI6, AOIn55w110, AOI8,
            AOI9, AOI10, AOI11, AOI12, AOIn50w95, AOI14, AOIn45w80, AOIn60w115, 
            AOIn65w90, AOIn65w95, AOIn50w80, AOIn50w85, AOIn55w100]

# Parametersn05
plot_maps = True
verbose = False

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# Peatland parameters
# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
update_shape_df = False
peatland_param_file = "peatland_params.json"

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# Landsat 8 parameters
# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# Prepare LANDSAT data ? 
prepare_landsat_data = False
aggreate_landsat_data = True

# Define the dates range
ls8_start_date = '2013-01-01'    
ls8_end_date = '2023-11-30' 

# Define the months range
ls8_start_month = 5
ls8_end_month   = 9  # before the snow  



In [ ]:
# Plot the AOI on a map
if plot_maps:
    import folium # the folium library
    world_map = folium.Map(zoom_start=2)

    # Iterator for colors
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 
              'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # add a rectangle to the map

    for AOI in list_AOI:
        folium.Rectangle(
            bounds=[(AOI["ymin"], AOI["xmin"]), (AOI["ymax"], AOI["xmax"])],
            fill=True,
            
            fill_opacity=0.2,
        ).add_to(world_map)
        # add the AOI to the map
        folium.Marker(
            location=[(AOI["ymin"]+AOI["ymax"])/2, (AOI["xmin"]+AOI["xmax"])/2],
            tooltip="AOI %s" % str(AOI),       
            icon=folium.Icon(icon="cloud"),
        ).add_to(world_map)

    world_map.fit_bounds(world_map.get_bounds())
    display(world_map)

## Load the LANDSAT data

If requested, the LANDSAT data are downloaded and prepared for the analysis. 
The data are downloaded from Google Earth Engine and first stored in your Google Drive.
You need to have a Google account and to have access to Google Earth Engine.

Then, after preparing the data, you need to retrieve them from your Google Drive and to store them in the data folder.

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
if prepare_landsat_data:
# Define the Earth Engine object, using the authentication credentials.
    # It will ask you to authenticate using your google account.
    # You will need to copy and paste a code in the terminal
    ee.Authenticate()

    # Initialize the Earth Engine object, using the authentication credentials.
    ee.Initialize()

    AOI_coord = [AOI["xmin"], AOI["ymin"], AOI["xmax"], AOI["ymax"]]
    AOI = ee.Geometry.Rectangle(AOI_coord, 'EPSG:4326', False)
 
    print("Start date: ", ls8_start_date)
    print("End date: ", ls8_end_date)

    print("Start month: ", ls8_start_month)
    print("End month: ", ls8_end_month)
    
    # Extract the data from the LANDSAT8 images from Google Earth Engine
    extract_ls8_data(AOI, ls8_start_date, ls8_end_date,
                        ls8_start_month, ls8_end_month)
    
else:
    print("LANDSAT8 data already extracted")
    print("Don't forget to retrieve the data from Google Earth Engine") 

### Aggregate the bands
The following cell aggregate all the file with different spectral band for a same month into a single file


In [ ]:
if aggreate_landsat_data:
    # Define the input and output folders
    LANDSAT_DIR = "/home/gsainton/Documents/01_Observatoire/Teledetection/LANDSAT8_DATA"
    input_folder = os.path.join(LANDSAT_DIR, "LS8_raw")
    output_folder = os.path.join(LANDSAT_DIR, "LS8_welldone")
    # Aggregate the data from the LANDSAT8 images

    landsat_aggregate_per_month(input_folder, output_folder)

In [ ]:
sanity_check = True

if sanity_check:
    # Get the list of files in the output folder
    files = os.listdir(output_folder)
    print("Number of files: ", len(files))
    print("First file: ", files[0])
    print("Last file: ", files[-1])
    
    
    #
    

## Path to the different folders and files used across the notebook <a class="anchor" id="allpaths"></a>

In [ ]:
# All the paths are stored in a JSON file named peatland_params.json
# This file is stored in the same directory as the notebook
# The JSON file is loaded and the paths are stored in variables

# Beware that there is no param file in the repo.

# The JSON file is structured as follows:
# {
#     "PEATMAP": "path/to/PEATMAP",
#     "LS8": "path/to/LS8",
#     "MERIT": "path/to/MERIT",
#     "WTD": "path/to/WTD"
# }

# Load the JSON file
with open(peatland_param_file) as json_file:
    datadir = json.load(json_file)

PEATMAP_DATA_DIR = datadir["PEATMAP"]
PEATMAP_DATA_DIR_WD = datadir["PEATMAP_WD"]
LS8_DATA_DIR = datadir["LS8"]
LS8_DATA_DIR_WD = datadir["LS8_WD"]
MERIT_DATA_DIR = datadir["MERIT"]
MERIT_DATA_DIR_WD = datadir["MERIT_WD"]
WTD_DATA_DIR = datadir["WTD"]
WTD_DATA_DIR_WD = datadir["WTD_WD"]

print("PEATMAP data directory: ", PEATMAP_DATA_DIR)
print("PEATMAP data directory (output directory): ", PEATMAP_DATA_DIR_WD)
print("LANDSAT8 data directory: ", LS8_DATA_DIR)
print("LANDSAT8 data directory (output directory): ", LS8_DATA_DIR_WD)
print("MERIT data directory: ", MERIT_DATA_DIR)
print("MERIT data directory (output directory): ", MERIT_DATA_DIR_WD)
print("WTD data directory: ", WTD_DATA_DIR)
print("WTD data directory (output directory): ", WTD_DATA_DIR_WD)


## Unzip PEATMAP files <a class="anchor" id="filemanip"></a>

In the next cell, we just unzip all the files .zip, to access the shape files.
Data from the PEATMAP article come from : https://archive.researchdata.leeds.ac.uk/251/

- All the zip files can be loaded into your `PEATMAP_DATA_DIR` 
- `unzip_files` function as its name says, unzip all the file in directories according to the location.
- Zipped files are moved into a directory `zip_files`. 

In [ ]:
# Unzip the files in the directory
list_zip_files = unzip_files(PEATMAP_DATA_DIR)

## Shape files to  Dataframe <a class="anchor" id="shp2df"></a>

Fonction to read all the shapefile and create a dataframe and a XML file to save all the information 

In [ ]:
# check if shapefiles_bounding_box.xml exists
if os.path.exists(os.path.join(PEATMAP_DATA_DIR, "shapefiles_bounding_box.xml")):
    print("shapefiles_bounding_box.xml already exists")
    if update_shape_df:
        print("Update the file requested")
        df_shapefiles = shapesfile2df(PEATMAP_DATA_DIR, verbose=verbose, export=True,
                              export_path=PEATMAP_DATA_DIR, export_name="shapefiles_bounding_box.xml")
        print(df_shapefiles)
else:

    df_shapefiles = shapesfile2df(PEATMAP_DATA_DIR, verbose=verbose, export=True,
                              export_path=PEATMAP_DATA_DIR, export_name="shapefiles_bounding_box.xml")
    display(df_shapefiles)

## Authentication to Google Earth Engine <a class="anchor" id="gee_auth"></a>

In [ ]:
import ee

ee.Authenticate()

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

## Select the shapefile from an given AOI <a class="anchor" id="select_shp"></a>

In [ ]:
# Check if the shapefiles_bounding_box.xml exists
if os.path.exists(os.path.join(PEATMAP_DATA_DIR, "shapefiles_bounding_box.xml")):
    df_shapefiles = pd.read_xml(os.path.join(PEATMAP_DATA_DIR, "shapefiles_bounding_box.xml"))
    display(df_shapefiles)
else:
    print("shapefiles_bounding_box.xml does not exist")

### Plot all the shapefile on a google map <a class="anchor" id="plotallshp"></a>

In [ ]:
if plot_maps:
    plot_all_shapefiles(df_shapefiles)


### Function to check the coverage of the shape files <a class="anchor" id="chkcoverage"></a>

Which files covers the best, a given AOI

In [ ]:
# Check if the zone is covered by one of the shapefiles
covered, dict_zone = check_zone_coverage(AOI, df_shapefiles)

# Plot the zone if covered
if covered and plot_maps:
    print("Zone name: ", dict_zone["zone_name"])
    print("Shapefile name: ", dict_zone["file_name"])
    print("Shapefile path: ", dict_zone["file_path"])
    print("Surface: ", dict_zone["surface"])

    # Open the shapefile and print the content the bounding box
    geozone = gpd.read_file(dict_zone["file_path"])
    geozone = geozone.to_crs("EPSG:4326")

    # Plot the shapefile with projection WGS 84
    fig, ax = plt.subplots(figsize=(10, 10))
    geozone.plot(ax=ax, color='green')
    ax.set_title("Map of PEATLANDS in " + dict_zone["zone_name"])
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    #ax.set_xlim(geozone.total_bounds[0], geozone.total_bounds[2])
    #ax.set_ylim(geozone.total_bounds[1], geozone.total_bounds[3])
    # grid
    ax.grid()

    # Add the rectangle patch of the AOI_hudson_bay
    from matplotlib.patches import Rectangle
    rect = Rectangle((AOI["xmin"], AOI["ymin"]),
                    AOI["xmax"] - AOI["xmin"],
                    AOI["ymax"] - AOI["ymin"],
                    linewidth=1, edgecolor='r', 
                    facecolor='none')
    ax.add_patch(rect)

    # Add a subplot with the zoom on the AOI_hudson_bay
    axins = ax.inset_axes([0.6, 0.6, 0.35, 0.35])

    geozone.plot(ax=axins, color='red',
                edgecolor='black', alpha=0.5)
    axins.set_xlim(AOI["xmin"], AOI["xmax"])
    axins.set_ylim(AOI["ymin"], AOI["ymax"])
    #axins.set_xticklabels('')
    #axins.set_yticklabels('')
    axins.grid()
    ax.indicate_inset_zoom(axins)


    plt.tight_layout()
    plt.show()
else:
    print("Zone not covered by the PEATLAND shapefiles")

### Function to crop the shape file to the AOI <a class="anchor" id="cropshp"></a>

In [ ]:
# Crop the shapefile
cropped_shapefile = crop_shapefile(geozone, AOI, plot=plot_maps, export=True, 
                                   export_path=PEATMAP_DATA_DIR_WD, 
                                   export_name=f"cropped_shapefile_{dict_zone['zone_name']}.shp")

# Plot the cropped shapefile
if plot_maps:
    fig, ax = plt.subplots(figsize=(10, 10))
    cropped_shapefile.plot(ax=ax, color='green')
    ax.set_title("ROI of the map of PEATLANDS in Canada")
    ax.set_xlabel("Longitude in degrees")
    ax.set_ylabel("Latitude in degrees")
    ax.set_xlim(cropped_shapefile.total_bounds[0], cropped_shapefile.total_bounds[2])
    ax.set_ylim(cropped_shapefile.total_bounds[1], cropped_shapefile.total_bounds[3])
    ax.grid()
    plt.show()


## Rasterisation shapefile <a class="anchor" id="rastershp"></a>

Rasterisation of the cropped shapefile the same LANDSAT8 resolution

All the LANSAT8 images must be retrieved from your Google Drive manually. 
In the next, we take one of these images as example for its dimension to rasterize the PEATMAP shapefile. 



In [ ]:

ls8_file_model = os.path.join(LS8_DATA_DIR, "LS8_B1_5.0.tif")

# Open the LS8 file 
with rasterio.open(ls8_file_model) as src:
    resolution = src.res

pixel_size = resolution[0]

# Rasterize each column of the shapefile into a separate raster file
shp2rasterfile = os.path.join(PEATMAP_DATA_DIR_WD, f"shp2raster_{dict_zone['zone_name']}.tif")
rasterize_shapefile(cropped_shapefile, src, 
                    os.path.join(PEATMAP_DATA_DIR_WD, shp2rasterfile), 
                    pixel_size, attribute="ALL",
                    verbose=verbose, control=True)


In [ ]:
path = "/home/gsainton/CALER/PEATMAP/0_data_preprocessing/1_GEE_DP/LS8_raw/LS8_n45w100"
path  = "/home/gsainton/CALER/PEATMAP/0_data_preprocessing/1_GEE_DP/LS8_raw/LS8_n55w105"

filename  = "LS8_n55w105_B1_5.tif"

# Open the LS8 file
with rasterio.open(os.path.join(path, filename)) as src:
    resolution = src.res

print("Resolution: ", resolution)

# Print bounds 
print("Bounds: ", src.bounds)


## Cell to check if the tif files have the right coverage

In [ ]:
import os
from glob import glob
import rasterio
from termcolor import colored

path = "/home/gsainton/CALER/PEATMAP/0_data_preprocessing/1_GEE_DP/LS8_raw/"

list_all_files = [f for f in glob(os.path.join(path, "**", "*.tif"), recursive=True)]
print("Number of files: ", len(list_all_files))

for f in list_all_files:
    if "B1_5" in f and "Hudson" not in f:
        print(f)
        # Extract coordinates from the filename
        coords = f.split("/")[-1].split("_")[1]
        #print("Coordinates: ", coords)
        coords = coords.split("n")[1]
        #print("Coordinates: ", coords)
        lat = float(coords.split("w")[0])
        lon = float(coords.split("w")[1])
        print("Latitude: ", lat)
        print("Longitude: ", lon)

        # Open the file and print the coordinate    
        with rasterio.open(f) as src:
            resolution = src.res
            if src.bounds.left - abs(lon) < 0.1 and src.bounds.right - abs(lon) < 0.1:
                
                # If ok, print with green color
                #print("OK : File is in the AOI")
                print(colored("OK : File is in the AOI", "green"))



            else:
                print(colored("KO : File is not in the AOI", "red"))
        # close file
        src.close()




# Check the coherency of the generated matlab files

In [ ]:
# dist0005, dist0100, dist1000, hand0005, hand0100, hand1000, 
# slope, elevation, wtd
# landsat 1, landsat 2, landsat 3, landsat 4, landsat 5, landsat 6, landsat 7
# permwater


import os
from glob import glob


path = "/home/gsainton/CALER/PEATMAP/1_NN_training/training_data/"

tile2load = "n50w95"

full_path = os.path.join(path, "trainingData_"+tile2load+".mat")
print("Full path: ", full_path)

# Check if the file exists
import h5py

# Load the file using h5py
with h5py.File(full_path, 'r') as file:
    # Access the data using the appropriate keys
    print("Keys: ", list(file.keys()))


# Plot the fist image
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import copy
import pandas as pd


# Load the file
with h5py.File(full_path, 'r') as mat_contents:

    data = mat_contents['input']
    print("Data shape: ", data.shape)

    target = mat_contents['target']

    long = mat_contents['lonS']
    lat = mat_contents['latS']

    # convert lat and long to numpy arrays
    lat = np.array(lat)
    long = np.array(long)
    
    print("Min lat: ", np.min(lat))
    print("Max lat: ", np.max(lat))
    print("Min long: ", np.min(long))
    print("Max long: ", np.max(long))

    df_data = pd.DataFrame(data)
    df_target = pd.DataFrame(target)

del data, target
mat_contents.close()


In [ ]:
print("**** INPUT DATA ****")
df_data_T = df_data.T
print(df_data_T.shape)

col_names = ["dist0005", "dist0100", "dist1000", "hand0005", "hand0100", "hand1000",
                "slope", "elevation", "wtd", "ls8_m7_b1", "ls8_m7_b2", "ls8_m7_b3",
                "ls8_m7_b4", "ls8_m7_b5", "ls8_m7_b6", "ls8_m7_b7", "NDVI"]

df_data_T.columns = col_names
# Add lat and long to the dataframe
df_data_T["lat"] = np.transpose(lat)
df_data_T["long"] = np.transpose(long)
del lat, long
display(df_data_T.describe())


print("**** TARGET ****")

df_target_T = df_target.T
col_names_target = ["peatmap"]
df_target_T.columns = col_names_target
display(df_target_T.describe())



In [ ]:
# Plot the peatmap, elevation, wtd, ls8_m7_b4, ls8_m7_b6 and NDVI in function of long and lat
# Create a subplot with 3 rows and 3 columns
fig, axs = plt.subplots(3, 3, figsize=(15, 15))

# remove all axis
for ax in axs.flat:
    ax.axis('off')


# Plot the peatmap
fig00 = axs[0, 0].scatter(df_data_T["long"], df_data_T["lat"], c=df_target_T["peatmap"], cmap='viridis')
axs[0, 0].set_title("Peatmap as target")
axs[0, 0].set_xlabel("Longitude")
axs[0, 0].set_ylabel("Latitude")
axs[0, 0].axis('off')
cbar_peat = fig.colorbar(fig00, ax=axs[0, 0])
cbar_peat.set_label("Peatmap percentage (%)")

# Plot the ls8_m7_b4
fig10 = axs[1, 0].scatter(df_data_T["long"], df_data_T["lat"], c=df_data_T["ls8_m7_b4"], cmap='grey')
axs[1, 0].set_title("LS8_M7_B4")
axs[1, 0].set_xlabel("Longitude")
axs[1, 0].set_ylabel("Latitude")
cbar_ls8_m7_b4 = fig.colorbar(fig10, ax=axs[1, 0])
cbar_ls8_m7_b4.set_label("Surface Reflectance")

# Plot the ls8_m7_b6
fig11 = axs[1, 1].scatter(df_data_T["long"], df_data_T["lat"], c=df_data_T["ls8_m7_b6"], cmap='grey')
axs[1, 1].set_title("LS8_M7_B6")
axs[1, 1].set_xlabel("Longitude")
axs[1, 1].set_ylabel("Latitude")
cbar_ls8_m7_b6 = fig.colorbar(fig11, ax=axs[1, 1])
cbar_ls8_m7_b6.set_label("Surface Reflectance")

# Plot the NDVI
fig12 = axs[1, 2].scatter(df_data_T["long"], df_data_T["lat"], c=df_data_T["NDVI"], cmap='viridis')
axs[1, 2].set_title("NDVI")
axs[1, 2].set_xlabel("Longitude")
axs[1, 2].set_ylabel("Latitude")
cbar_ndvi = fig.colorbar(fig12, ax=axs[1, 2])
cbar_ndvi.set_label("NDVI")

# Plot the elevation
fig20 = axs[2, 0].scatter(df_data_T["long"], df_data_T["lat"], c=df_data_T["elevation"], cmap='viridis')
axs[2, 0].set_title("Elevation")
axs[2, 0].set_xlabel("Longitude")
axs[2, 0].set_ylabel("Latitude")
cbar_elevation = fig.colorbar(fig20, ax=axs[2, 0])
cbar_elevation.set_label("Elevation [m]")

# Plot the wtd
fig21 = axs[2, 1].scatter(df_data_T["long"], df_data_T["lat"], c=df_data_T["wtd"], cmap='viridis')
axs[2, 1].set_title("WTD")
axs[2, 1].set_xlabel("Longitude")
axs[2, 1].set_ylabel("Latitude")
cbar_wtd = fig.colorbar(fig21, ax=axs[2, 1])
cbar_wtd.set_label("Water Table Depth [m]")

# Distance to large drainage
fig22 = axs[2, 2].scatter(df_data_T["long"], df_data_T["lat"], c=df_data_T["dist1000"], cmap='viridis')
axs[2, 2].set_title("Distance to large drainage")
axs[2, 2].set_xlabel("Longitude")
axs[2, 2].set_ylabel("Latitude")
cbar_dist1000 = fig.colorbar(fig22, ax=axs[2, 2])
cbar_dist1000.set_label("[km]")

plt.tight_layout()
plt.show()
plt.savefig(f"peatmap_data_{tile2load}.png")


In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show

# Tif files directory
path = "/home/gsainton/CALER/PEATMAP/0_data_preprocessing/1_GEE_DP/LS8_raw/"

# List all the tif files
list_all_files = [f for f in glob(os.path.join(path, "**", "*.tif"), recursive=True)]

tile2load = "n50w95"

# Check files containing the tile2load
list_files = []
for f in list_all_files:
    if tile2load in f:
        list_files.append(f)

# Load the first file
filename = list_files[0]

# Open the file
with rasterio.open(filename) as src:
    # Get the resolution
    resolution = src.res
    # Get the bounds
    bounds = src.bounds
    # Get the shape
    shape = src.shape
    # Get the number of bands
    nbands = src.count
    # Get the crs
    crs = src.crs
    # Get the transform
    transform = src.transform
    # Plot the image in degrees coordinates
    fig, ax = plt.subplots(figsize=(10, 10))

    show(src, ax=ax)

    ax.set_title("LS8 image in degrees coordinates")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.show()

